In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(font_scale=1.5)

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [2]:
df = pd.read_csv('poly_ewm18_unstructured.csv', index_col='Unnamed: 0')
variables = []
for col in df.columns:
    if 'id' in col:
        df[col] = df[col].astype('category')

for col in df.columns:
    if df[col].dtype == float or df[col].dtype == int:
        if 'difference' not in col:
            variables.append(col)
print('Numerical features :\n', variables)        

Numerical features :
 ['location', 'type', 'outcome', '1', 'height', 'ewm_seconds', 'ewm_fgm', 'ewm_fga', 'ewm_fg3m', 'ewm_fg3a', 'ewm_ftm', 'ewm_fta', 'ewm_oreb', 'ewm_dreb', 'ewm_reb', 'ewm_ast', 'ewm_stl', 'ewm_blk', 'ewm_to', 'ewm_pf', 'ewm_pts', 'ewm_poss', 'ewm_off_rate', 'ewm_def_rate', 'ewm_net_rate', 'ewm_pace', 'ewm_ast_pct', 'ewm_ast_tov', 'ewm_ast_ratio', 'ewm_oreb_pct', 'ewm_dreb_pct', 'ewm_reb_pct', 'ewm_tm_tov_pct', 'ewm_usg_pct', 'ewm_game_score', 'ewm_plus_minus', 'height^2', 'height ewm_seconds', 'height ewm_fgm', 'height ewm_fga', 'height ewm_fg3m', 'height ewm_fg3a', 'height ewm_ftm', 'height ewm_fta', 'height ewm_oreb', 'height ewm_dreb', 'height ewm_reb', 'height ewm_ast', 'height ewm_stl', 'height ewm_blk', 'height ewm_to', 'height ewm_pf', 'height ewm_pts', 'height ewm_poss', 'height ewm_off_rate', 'height ewm_def_rate', 'height ewm_net_rate', 'height ewm_pace', 'height ewm_ast_pct', 'height ewm_ast_tov', 'height ewm_ast_ratio', 'height ewm_oreb_pct', 'height ew

In [3]:
grouped_data = df.groupby(['game_id', 'team_id'])

# 20000011 ['1610612744', '1610612756']

t2 = grouped_data.get_group((20000011, 1610612744)).sort_values(by='ewm_seconds', ascending=False)
t3 = grouped_data.get_group((20000011, 1610612756)).sort_values(by='ewm_seconds', ascending=False)

a = np.vstack((np.array(t2[t2['start_position']=='G'].sort_values(by='height', ascending=False)),
         np.array(t2[t2['start_position']=='F'].sort_values(by='height', ascending=False)),
          np.array(t2[t2['start_position']=='C']),
          np.array(t2[t2['start_position'].isin(
              ['G','F','C'])==False].sort_values(
              by='ewm_seconds', ascending=False).iloc[:3,:].sort_values(
          by='height')))).ravel()

b = np.vstack((np.array(t3[t3['start_position']=='G'].sort_values(by='height', ascending=False)),
         np.array(t3[t3['start_position']=='F'].sort_values(by='height', ascending=False)),
          np.array(t3[t3['start_position']=='C']),
          np.array(t3[t3['start_position'].isin(
              ['G','F','C'])==False].sort_values(
              by='ewm_seconds', ascending=False).iloc[:3,:].sort_values(
          by='height')))).ravel()

game_test_columns = np.array(
    [['{}_{}'.format(i, col) for col in t2.columns] for i in range(1,9)]).ravel()

game_test = pd.DataFrame(np.vstack(
    (a,b)), columns=game_test_columns)
game_test

,1_date,1_season_id,1_location,1_type,1_outcome,1_difference,1_game_id,1_team_id,1_team_abb,1_team_city,...,8_ewm_tm_tov_pct^2,8_ewm_tm_tov_pct ewm_usg_pct,8_ewm_tm_tov_pct ewm_game_score,8_ewm_tm_tov_pct ewm_plus_minus,8_ewm_usg_pct^2,8_ewm_usg_pct ewm_game_score,8_ewm_usg_pct ewm_plus_minus,8_ewm_game_score^2,8_ewm_game_score ewm_plus_minus,8_ewm_plus_minus^2
0,2000-10-31,22000,1,1,1,2,20000011,1610612744,GSW,Golden State,...,99.5681,1.14902,75.0806,-97.3218,0.0132599,0.866437,-1.1231,56.6155,-73.3867,95.1261
1,2000-10-31,22000,0,1,0,2,20000011,1610612756,PHX,Phoenix,...,454.478,2.83878,71.195,-5.84392,0.0177317,0.444702,-0.0365026,11.1529,-0.915464,0.0751442


In [4]:
def team_ravel(df): 
    return np.vstack((np.array(df[df['start_position']=='G'].sort_values(by='height', ascending=False)),
         np.array(df[df['start_position']=='F'].sort_values(by='height', ascending=False)),
          np.array(df[df['start_position']=='C']),
          np.array(df[df['start_position'].isin(
              ['G','F','C'])==False].sort_values(
              by='ewm_seconds', ascending=False).iloc[:3,:].sort_values(
          by='height')))).ravel()

In [5]:
def team_ravel_bymin(df):
    return np.array(df.sort_values(by='ewm_seconds')).ravel()

In [7]:
remove = ['date', 'season_id', 'location', 'type', 'outcome','difference',
          'game_id', 'team_id', 'team_abb','team_city','start_position']

necessary_drop = [col for col in game_test.loc[:,'1_name':].columns if \
                  any(n in col for n in remove)]

game_test.drop(columns=necessary_drop, inplace=True)

end_game = np.array([['{}{}'.format(ha, col) for col in game_test.columns] \
                    for ha in ['h_','a_']]).ravel()

In [8]:
len(end_game)

9044

In [9]:
game_keys = {}
for gameid, teamid in grouped_data.groups.keys():
    game_keys[gameid] = []
for gameid, teamid in grouped_data.groups.keys():
    game_keys[gameid].append(teamid)

In [12]:
from tqdm import tqdm

all_rows = np.array([])
reversed_rows = np.array([])
c=0
e=0
errors = {}
for game in tqdm(list(game_keys.keys())):
    if game == 0:
        continue
    try:
        a = grouped_data.get_group((game, game_keys[game][0]))
        b = grouped_data.get_group((game, game_keys[game][1]))
        a = team_ravel_bymin(a)
        b = team_ravel_bymin(b)
        game_df = pd.DataFrame(np.vstack(
            (a,b)), columns=game_test_columns)
        c+=1
    except:
        errors[game] = game_keys[game]
        e+=1
        continue
#         print(f'Total errors: {e}')
    
    remove = ['date', 'season_id', 'location', 'type', 'outcome','difference',
          'game_id', 'team_id', 'team_abb','team_city','start_position']

    necessary_drop = [col for col in game_df.loc[:,'1_name':].columns if \
                  any(n in col for n in remove)]
    
    game_df.drop(columns=necessary_drop, inplace=True)
    game_df['1_location'] = game_df['1_location'].apply(lambda x: int(x))
    
    row = np.array(
        game_df.sort_values(by='1_location',
                            ascending=False)).ravel()
#     reversed_row = np.array(
#         game_df.sort_values(by='pg_location')).ravel()
    
    try:
        if all_rows.size == 0:
            all_rows = np.vstack([row])
#             reversed_rows = np.vstack([reversed_row])
        else:
            all_rows = np.vstack([all_rows, row])
#             reversed_rows = np.vstack(
#                 [reversed_rows, reversed_row])
    except:
#         print('\n===ERROR!!!===\n', game, bi_keys[game], '\n===!!!!!!!!===\n')
        pass

structured_df = pd.DataFrame(all_rows,
                             columns=end_game)
# reversed_df = pd.DataFrame(reversed_rows,
#                            columns=end_game)
display(structured_df.head())
# display(reversed_df.head())
print(structured_df.shape)
print(f'TOTAL ERRORS  : {e}')
print(f'ERROR PERCENT : {(e/(e+c) * 100):0f}%')

100%|██████████| 26936/26936 [56:33<00:00,  7.94it/s]  


,h_1_date,h_1_season_id,h_1_location,h_1_type,h_1_outcome,h_1_difference,h_1_game_id,h_1_team_id,h_1_team_abb,h_1_team_city,...,a_8_ewm_tm_tov_pct^2,a_8_ewm_tm_tov_pct ewm_usg_pct,a_8_ewm_tm_tov_pct ewm_game_score,a_8_ewm_tm_tov_pct ewm_plus_minus,a_8_ewm_usg_pct^2,a_8_ewm_usg_pct ewm_game_score,a_8_ewm_usg_pct ewm_plus_minus,a_8_ewm_game_score^2,a_8_ewm_game_score ewm_plus_minus,a_8_ewm_plus_minus^2
0,2000-11-07,22000,1,1,0,-14,20000057,1610612749,MIL,Milwaukee,...,88.2159,2.25485,103.73,-20.7304,0.0576354,2.65141,-0.529882,121.973,-24.3762,4.87157
1,2000-11-15,22000,1,1,1,2,20000116,1610612750,MIN,Minnesota,...,152.013,3.08911,178.127,-80.7372,0.0627748,3.61977,-1.64069,208.726,-94.6067,42.8812
2,2000-11-17,22000,1,1,0,-5,20000132,1610612756,PHX,Phoenix,...,87.3035,2.21517,91.8264,-14.4651,0.0562057,2.32993,-0.367025,96.5836,-15.2145,2.39669
3,2000-11-25,22000,1,1,0,-4,20000182,1610612752,NYK,New York,...,68.199,2.45819,170.672,37.4905,0.0886042,6.15179,1.35133,427.119,93.8226,20.6094
4,2000-11-28,22000,1,1,1,14,20000204,1610612742,DAL,Dallas,...,145.638,1.77851,74.3427,61.0686,0.0217188,0.90786,0.74576,37.9491,31.1732,25.6072


(731, 9044)
TOTAL ERRORS  : 26205
ERROR PERCENT : 97.286160%


In [13]:
structured_df.to_csv('poly_ewm18_seconds.csv')